In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import copy
import sys

sys.path.append('./mcunet')

from mcunet.gumbel_module.gumbel_net import GumbelMCUNets
from mcunet.gumbel_module.gumbel_layer import MBGumbelInvertedConvLayer, MobileGumbelInvertedResidualBlock
from mcunet.tinynas.nn.modules import MBInvertedConvLayer
from mcunet.tinynas.nn.networks import MobileInvertedResidualBlock
from mcunet.model_zoo import build_model

from mcunet.utils import MyModule, MyNetwork, SEModule, build_activation, get_same_padding, sub_filter_start_end
from mcunet.tinynas.nn.modules import ZeroLayer, set_layer_from_config

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ori_model, img_size, desc = build_model(net_id='mcunet-in4', pretrained=True)
gubmel_config = {'global_expand_ratio_list':[1,3,4,5,6], 'global_kernel_size_list':[3,5,7], 'gumbel_feature_extract_block':2}
gumbel_model = GumbelMCUNets.build_from_config(ori_model.config, gubmel_config)
gumbel_model.load_pretrained_mcunet_param(ori_model)

inputs = torch.randn(16, 3, 160, 160)
gumbel_model = gumbel_model.cuda()
output, gumbel_list = gumbel_model.forward(inputs.cuda())
#out = gumbel_model.forward_original(inputs)
#out2 = ori_model.forward(inputs)
#print((out - out2).sum())
#gumbel_model.set_static_flops(inputs)

load pretrained mcumodel to gumbel net
tensor([[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.]], device='cuda:0', grad_fn=<AddBackward0>)
0 tensor(1, device='cuda:0')
1 tensor(3, device='cuda:0')
2 tensor(4, device='cuda:0')
tensor([[1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1.],
        [0., 1., 0., 0., 1.],
        [1., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0.],
        [0., 1., 0., 1., 0.],
        [0., 0., 1., 0., 1.],
        [0., 1., 0., 0., 1.],
        [0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [0., 0., 1., 0., 1

In [3]:
gumbel_model.set_static_flops(inputs.cuda())
gumbel_model.compute_flops(inputs, gumbel_list)

Unsupported operator aten::add_ encountered 1 time(s)
Unsupported operator aten::hardtanh_ encountered 1 time(s)
Unsupported operator aten::hardtanh_ encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
mobile_inverted_conv.depth_conv.bn, mobile_inverted_conv.point_linear.bn
Unsupported operator aten::hardtanh_ encountered 2 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
mobile_inverted_conv.depth_conv.bn, mobile_inverted_

Success Log Static & Dynamic Flops : 367427584, 1670131200
initialize FLOPs table
expand ratio list > 1, kernel size list = 1
inverted blocks flops : tensor([ 921600., 2764800., 3686400., 4608000.], device='cuda:0')
dw blocks flops : tensor([14400., 43200., 57600., 72000.], device='cuda:0')
pw blocks flops : tensor([ 4608000., 13824000., 18432000., 23040000.], device='cuda:0')
initialize FLOPs table
expand ratio list > 1 (3), kernel size list > 1 (2)
inverted blocks flops : tensor([ 921600., 2764800., 3686400.], device='cuda:0')
dw convert flops :,  tensor([   0., 7776.], device='cuda:0')
dw blocks flops : tensor([3840000., 1382400.], device='cuda:0')
pw blocks flops : tensor([ 3686400., 11059200., 14745600.], device='cuda:0')
initialize FLOPs table
expand ratio list > 1 (4), kernel size list > 1 (3)
inverted blocks flops : tensor([ 921600., 2764800., 3686400., 4608000.], device='cuda:0')
dw convert flops :,  tensor([    0., 75000., 84720.], device='cuda:0')
dw blocks flops : tensor([1

tensor([2.9094e+08, 2.7615e+08, 2.8728e+08, 2.6650e+08, 1.8582e+08, 2.1202e+08,
        2.3700e+08, 2.2872e+08, 2.7107e+08, 2.6607e+08, 3.1530e+08, 2.5496e+08,
        1.7824e+08, 2.4217e+08, 2.1952e+08, 2.4054e+08], device='cuda:0',
       grad_fn=<AddBackward0>)

In [8]:
torch.tensor([1,2,3,4])* torch.tensor(block.mobile_inverted_conv.expand_ratio_list)

tensor([ 1,  6, 12, 20])

In [9]:
[1,2,3,4] * [2,6,8,12]

TypeError: can't multiply sequence by non-int of type 'list'

In [4]:
for i, block in enumerate(gumbel_model.blocks):
    if isinstance(block, MobileGumbelInvertedResidualBlock):
        print(block.shortcut, block.mobile_inverted_conv.depth_conv.conv.stride)
        break

IdentityLayer() (1, 1)


In [ ]:
gumbel_model.forward(inputs)

gumbel_model.eval()
print(gumbel_model.training)
out_origin = gumbel_model.forward(inputs)
for i in range(10):
    print(out_origin - gumbel_model.forward(inputs))


In [ ]:
class test_gumbel(nn.Module):
    def __init__(self):
        super().__init__()
        self.test_layer = nn.Linear(20, 4)
    
    def forward(self, x):
        gumbel_input = self.test_layer(x)
        if self.training:
            gumbel_out = F.gumbel_softmax(gumbel_input, tau=1, hard=True, eps=1e-10, dim=-1)
        else:
            index = gumbel_input.max(dim=-1, keepdim=True)[1]
            gumbel_out = torch.zeros_like(gumbel_input, memory_format=torch.legacy_contiguous_format).scatter_(-1, index, 1.0)
        
        return gumbel_out
inputs = torch.randn(5, 20)
gumbel = test_gumbel()

# train
gumbel.train()
for i in range(5):
    out = gumbel(inputs)
    print(f"{i} iter -> ", out)

# test
gumbel.eval()
for i in range(5):
    out = gumbel(inputs)
    print(f"{i} iter -> ", out)


In [ ]:
from torchprofile import profile_macs

total_mac = profile_macs(gumbel_model.cuda(), torch.randn(2, 3,160, 160).cuda())
print(total_mac)

In [ ]:
from torchprofile.utils.flatten import Flatten
import warnings
with warnings.catch_warnings(record=True):
    graph, _ = torch.jit._get_trace_graph(Flatten(gumbel_model.cuda()), torch.randn(2,3,160,160).cuda(), None)

In [ ]:
variables = dict()
for x in graph.nodes():
    for v in list(x.inputs()):
        if 'tensor' in v.type().kind().lower():
            print(v.debugName(), v.type().scalarType(), v.type().sizes())

In [ ]:
for x in graph.nodes():
    if 'mul_' in x.kind().lower():
        print(x)

In [ ]:
ori_model_size = sum([p.numel() for p in ori_model.parameters()]) * 4 / 2**20
gumbel_model_size = sum([p.numel() for p in gumbel_model.parameters()]) * 4 / 2**20
print("Ori model size : %.1f MB" % ori_model_size)
print("Gumbel model size : %.1f MB" % gumbel_model_size)

from torchinfo import summary

summary(gumbel_model, input_size=(4, 3, 160, 160), col_width=16, col_names=['kernel_size', 'output_size', 'num_params', 'mult_adds', 'params_percent'], depth=2)


In [ ]:
print("before forward grad : ", gumbel_model.gumbel_fc1.weight.grad)
out = gumbel_model(torch.randn(32, 3, 160, 160))

out.sum().backward()

print("after forward grad : \n", gumbel_model.gumbel_fc1.weight.grad)

In [ ]:
for n, p in net.named_parameters():
    if has_deep_attr(model, n):
        print(n)

In [ ]:
model

In [ ]:
mbconv_test = MBGumbelInvertedConvLayer.build_from_config(m.mobile_inverted_conv.config)
mbconv_test.config

In [ ]:
inputs = torch.randn(2, 16, 32, 32)
gumbel_inputs = torch.randn(2, 4, 8, 8)
gumbel_inputs.requires_grad = True
gumbel_layer = nn.Linear(4*8*8, 5)
gumbel_output = gumbel_layer(gumbel_inputs.view(2, -1))
gumbel_index = F.gumbel_softmax(gumbel_output, tau=1, hard=True)
print(gumbel_index)
out = mbconv_test.forward(torch.randn(2, 16, 32, 32))

In [ ]:
inputs = torch.randn(2, 16, 32, 32)
gumbel_inputs = torch.randn(2, 4, 8, 8)
gumbel_inputs.requires_grad = True
gumbel_layer = nn.Linear(4*8*8, 5)
gumbel_output = gumbel_layer(gumbel_inputs.view(2, -1))
gumbel_index = F.gumbel_softmax(gumbel_output, tau=1, hard=True)
print(gumbel_index)
out = mbconv_test.forward(torch.randn(2, 16, 32, 32), gumbel_index)
out.sum().backward()

In [ ]:
gumbel_layer.weight.grad

In [ ]:
original_mbconv_test_weight = copy.deepcopy(mbconv_test.depth_conv.conv.weight)
print(original_mbconv_test_weight)

In [ ]:
print(m.mobile_inverted_conv.depth_conv.conv.weight)

In [ ]:
for n, p in m.mobile_inverted_conv.named_parameters():
    if has_deep_attr(mbconv_test, n):
        print(n, p)
        set_deep_attr(mbconv_test, n, p)
        print('------------------')

In [ ]:
for n, p in m.mobile_inverted_conv.named_parameters():
    if has_deep_attr(mbconv_test, n):
        print(n)
        print(get_deep_attr(mbconv_test, n) - p)

In [ ]:
mbconv_test.forward(torch.randn(1,32,16,16), gumbel=1)

In [ ]:
bn_layer = nn.BatchNorm2d(16)

In [ ]:
x = torch.randn(1, 12, 32, 32)

In [ ]:
feature_dim = 12
out = F.batch_norm(x, bn_layer.running_mean[:feature_dim], bn_layer.running_var[:feature_dim], bn_layer.weight[:feature_dim], bn_layer.bias[:feature_dim])

In [ ]:
out.sum().backward()

In [ ]:
bn_layer.weight.grad

In [ ]:
model, img_size, desc = build_model(net_id='mcunet-in4', pretrained=True)

backup_model = copy.deepcopy(model)
model_copy = build_model(net_id='mcunet-in4', pretrained=False)[0]

for (n1, p1), (n2, p2) in zip(backup_model.named_parameters(), model_copy.named_parameters()):
    if n1 == n2:
        print((p1 - p2).sum())

In [ ]:
for n, p in model.named_parameters():
    if has_deep_attr(model_copy, n):
        print(n)
        set_deep_attr(model_copy, n, p)

In [ ]:
for (n1, p1), (n2, p2) in zip(backup_model.named_parameters(), model_copy.named_parameters()):
    if n1 == n2:
        print((p1-p2).sum())